In [ ]:
import glob
import os
import pandas as pd

!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-oscar-repetitions
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-c4-repetitions
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-oscarseeds
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-c4seeds
!mv lm1-2b8-55b-oscarseeds/* lm1-2b8-55b-oscar-repetitions/
!mv lm1-2b8-55b-c4seeds/* lm1-2b8-55b-c4-repetitions/

!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-c4-repetitions
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-c4seeds
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-oscar-repetitions
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-oscarseeds
!mv lm1-4b2-84b-oscarseeds/* lm1-4b2-84b-oscar-repetitions/
!mv lm1-4b2-84b-c4seeds/* lm1-4b2-84b-c4-repetitions/

!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-8b7-178b-c4-repetitions
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-8b7-178b-oscar-repetitions

!mkdir lm1-4b2-84b-c4-opt
!cp -r lm1-4b2-84b-c4-repetitions/*4b284b12bc4* lm1-4b2-84b-c4-opt/

TASK_TO_BASELINE = {
    "anli_r1": 1/3,
    "anli_r2": 1/3,
    "anli_r3": 1/3,
    "arc_challenge": 1/4,
    "arc_easy": 1/4,
    "boolq": 1/2,
    "cb": 1/3,
    "copa": 1/2,
    "hellaswag": 1/4,
    "piqa": 1/2,
    "rte": 1/2,
    "sciq": 1/4,
    "storycloze_2016": 1/4,
    "winogrande": 1/2,
    "babi": 0,
}


MODELS_MAP = {
   "lm1-2b8-55b-c4-repetitions": {
      44: "2b855b1b25c4",
      14: "2b855b4bc4",
      7: "2b855b9bc4",
	    5: "2b855b11bc4",
      4: "2b855b14bc4",
      3: "2b855b18bc4",
      2: "2b855b28bc4",
      1: "2b855b55bc4",
  },
  "lm1-2b8-55b-oscar-repetitions": {
      44: "2b855b1b25oscar",
      14: "2b855b4boscar",
      7: "2b855b9boscar",
	    5: "2b855b11boscar",
      4: "2b855b14boscar",
      3: "2b855b18boscar",
      2: "2b855b28boscar",
      1: "2b855b55boscar",
  },
  "lm1-4b2-84b-c4-repetitions": {
      44: "4b284b1b9c4",
	    14: "4b284b6bc4",
      7: "4b284b12bc4",
	    5: "4b284b17bc4",
      4: "4b284b21bc4",
      3: "4b284b28bc4",
      2: "4b284b42bc4",
      1: "4b284b84bc4v2",
  },
  "lm1-4b2-84b-c4-opt": {
      10: "4b284b12bc4opt4",
      7: "4b284b12bc4",
  },  
  "lm1-4b2-84b-oscar-repetitions": {
      44: "4b284b1b9oscar",
	    14: "4b284b6boscar",
      7: "4b284b12boscar",
	    5: "4b284b17boscar",
      4: "4b284b21boscar",
      3: "4b284b28boscar",
      2: "4b284b42boscar",
      1: "4b284b84boscarv2",
  },
  "lm1-8b7-178b-c4-repetitions": {
      44: "8b7178b4b",
      14: "8b7178b13b",
      7: "8b7178b25b",
	    5: "8b7178b35b",
      4: "8b7178b44b",
      3: "8b7178b58b",
      2: "8b7178b88b",
      1: "8b7178b178b",
   },
  "lm1-8b7-178b-oscar-repetitions": {
      44: "8b7178b4b",
      14: "8b7178b13b",      
      7: "8b7178b25b",
      5: "8b7178b35b",
      4: "8b7178b44b",
      3: "8b7178b58b",
      2: "8b7178b88b",
      1: "8b7178b178b",
  }
}

MODEL_TO_FEWSHOT_SCORES = {}
MODEL_TO_FEWSHOT_SCORES_ACC = {}
MODEL_TO_FEWSHOT_SCORES_GEN = {}

SHOTS = list(range(6))

for (BASE_MODEL, MODELS) in MODELS_MAP.items():
    MODELS = set(MODELS.values())

    MODEL_TO_FEWSHOT_SCORES.setdefault(BASE_MODEL, {})
    MODEL_TO_FEWSHOT_SCORES_ACC.setdefault(BASE_MODEL, {})
    MODEL_TO_FEWSHOT_SCORES_GEN.setdefault(BASE_MODEL, {})

    if os.path.isdir(BASE_MODEL.replace("-repetitions", "seeds")):
        !mv {BASE_MODEL.replace("-repetitions", "seeds")}/* {BASE_MODEL}/
        MODELS |= {f"{MODEL}seed{seed}" for MODEL in MODELS for seed in [1, 2, 3, 4]}

    for MODEL in MODELS:

        MODEL_TO_FEWSHOT_SCORES[BASE_MODEL].setdefault(MODEL, {})
        MODEL_TO_FEWSHOT_SCORES_ACC[BASE_MODEL].setdefault(MODEL, {})
        MODEL_TO_FEWSHOT_SCORES_GEN[BASE_MODEL].setdefault(MODEL, {})

        path = f"{BASE_MODEL}/{MODEL}/evaluation/generation/merged.csv"
        if not os.path.exists(path):
            print("Skipping: ", f"{BASE_MODEL}/{MODEL}")
            continue
        generation = pd.read_csv(path)
        for SHOT in SHOTS: 
              
            MODEL_TO_FEWSHOT_SCORES[BASE_MODEL][MODEL].setdefault(SHOT, {})
            MODEL_TO_FEWSHOT_SCORES_ACC[BASE_MODEL][MODEL].setdefault(SHOT, {})
            MODEL_TO_FEWSHOT_SCORES_GEN[BASE_MODEL][MODEL].setdefault(SHOT, {})

            rankeval_files = glob.glob(f"{BASE_MODEL}/{MODEL}/evaluation/rankeval/*_{SHOT}.csv")
            assert len(rankeval_files) == 1, f"{rankeval_files}"
            rankeval = pd.read_csv(rankeval_files[0])

            # Rescale to 0 - 1, where 0 is random performance
            rankeval["normalized"] = rankeval.apply(lambda x: (x["value"] - TASK_TO_BASELINE[x["task"]]) / (1 - TASK_TO_BASELINE[x["task"]]), axis=1)
            rankeval = rankeval[rankeval["metric"] == "acc"]
            rankeval_scores = rankeval.normalized.values.tolist()

            gen_sub = generation[generation["fewshots"] == SHOT]
            gen_sub = gen_sub[gen_sub["prompt"] != "median"]
            gen_sub = gen_sub[gen_sub["prompt"] != "average"]

            # 0 is already random performance, i.e. no rescaling necessary
            generation_scores = gen_sub.value.values.tolist()
            scores = rankeval_scores + generation_scores
            average_score = sum(scores) / len(scores)

            for i, row in rankeval.iterrows():
                MODEL_TO_FEWSHOT_SCORES[BASE_MODEL][MODEL][SHOT][row["task"]] = row["normalized"]
            for i, row in gen_sub.iterrows():
                MODEL_TO_FEWSHOT_SCORES[BASE_MODEL][MODEL][SHOT][row["dataset"]] = row["value"]

            babi_files = glob.glob(f"{BASE_MODEL}/{MODEL}/evaluation/*{SHOT}_babi.json")
            if len(babi_files) == 1:
                import json
                with open(babi_files[0], "r") as f:
                    MODEL_TO_FEWSHOT_SCORES[BASE_MODEL][MODEL][SHOT]["babi"] = json.load(f)["results"]["babi"]["em"]
            else:
                print(f"Missing bAbI: {BASE_MODEL}/{MODEL}; Setting to 0 (which is OK for C4)")
                MODEL_TO_FEWSHOT_SCORES[BASE_MODEL][MODEL][SHOT]["babi"] = 0

            #MODEL_TO_FEWSHOT_SCORES[BASE_MODEL][MODEL][SHOT]
            #MODEL_TO_FEWSHOT_SCORES_ACC[BASE_MODEL][MODEL].append(sum(rankeval_scores) / len(rankeval_scores))
            #MODEL_TO_FEWSHOT_SCORES_GEN[BASE_MODEL][MODEL].append(sum(generation_scores) / len(generation_scores))


Cloning into 'lm1-2b8-55b-oscar-repetitions'...
remote: Enumerating objects: 3335, done.
remote: Counting objects: 100% (3233/3233), done.
remote: Compressing objects: 100% (2806/2806), done.
remote: Total 3335 (delta 487), reused 3139 (delta 420), pack-reused 102
Receiving objects: 100% (3335/3335), 8.68 MiB | 11.66 MiB/s, done.
Resolving deltas: 100% (541/541), done.
Updating files: 100% (3232/3232), done.
Cloning into 'lm1-2b8-55b-c4-repetitions'...
remote: Enumerating objects: 4652, done.
remote: Counting objects: 100% (4223/4223), done.
remote: Compressing objects: 100% (3694/3694), done.
remote: Total 4652 (delta 562), reused 4143 (delta 518), pack-reused 429
Receiving objects: 100% (4652/4652), 10.27 MiB | 4.88 MiB/s, done.
Resolving deltas: 100% (635/635), done.
Updating files: 100% (4400/4400), done.
Cloning into 'lm1-2b8-55b-oscarseeds'...
remote: Enumerating objects: 12681, done.
remote: Counting objects: 100% (284/284), done.
remote: Compressing objects: 100% (79/79), done.

In [ ]:
### Latex Table ###

import numpy as np

TASK_TO_NAME = {
    "anli_r1": "ANLI R1",
    "anli_r2": "ANLI R2",
    "anli_r3": "ANLI R3",
    "arc_challenge": "ARC-Challenge",
    "arc_easy": "ARC-Easy",
    "boolq": "BoolQ",
    "cb": "CB",
    "copa": "COPA",
    "hellaswag": "HellaSwag",
    "piqa": "PiQA",
    "rte": "RTE",
    "sciq": "SciQ",
    "storycloze_2016": "StoryCloze 2016",
    "winogrande": "WinoGrande XL",

    "e2e_nlg_cleaned": "E2E NLG",
    "gem_xsum": "XSUM",
    "web_nlg_en": "WebNLG EN",
    "wiki_lingua_en": "WikiLingua EN",
    
    "babi": "bAbI",
}


TABLE = """
\\begin{table}
  \\centering
  \\caption{2.8B perf.}
  \\resizebox{\\textwidth}{!}{
    \\begin{tabular}{l|llllllll}
"""

MODEL_KEY = 'lm1-2b8-55b-oscar-repetitions' # Change to desired model
MODEL_KEY = 'lm1-4b2-84b-c4-repetitions' # Change to desired model


MODEL_FAM = list(MODELS_MAP[MODEL_KEY].values())[::-1]
MODEL_EPS = list(MODELS_MAP[MODEL_KEY].keys())[::-1]

SUBHEADER = "Unique Tokens & " + " & ".join([str(x) for x in MODEL_EPS]) + " \\\\" + "\n"
SUBSUBHEADER = "Epochs & " + " & ".join([str(x) for x in MODEL_EPS]) + " \\\\" + "\n"

TABLE += SUBHEADER + SUBSUBHEADER

for task in TASK_TO_NAME:
    scores_c4 = []
    std_c4 = []
    for model in MODEL_FAM:
        if "seed" in model: continue
        model_scores = []
        for model_name in [model+"", model+"seed1", model+"seed2", model+"seed3", model+"seed4"]:
            model_scores.append(sum([MODEL_TO_FEWSHOT_SCORES[MODEL_KEY][model_name][SHOT][task] for SHOT in [0, 1, 2, 3, 4, 5]]) / 6)
        scores_c4.append(np.mean(model_scores))
        std_c4.append(np.std(model_scores))

    scores_c4 = [f"{str(round(score * 100, 1))} ± {str(round(std * 100, 1))}" for score, std in zip(scores_c4, std_c4)]
    TABLE += "\n" + f"{TASK_TO_NAME[task]} & " + " & ".join(scores_c4) + " \\\\"

    if task in ("winogrande", "wiki_lingua_en", "babi"):
        TABLE += "\n" + "\\midrule"

# Add average
scores_avg_str_c4 = []
for model in MODEL_FAM:
    if "seed" in model: continue
    all_scores = []
    for model_name in [model+"", model+"seed1", model+"seed2", model+"seed3", model+"seed4"]:
        scores = [list(MODEL_TO_FEWSHOT_SCORES[MODEL_KEY][model_name][SHOT].values()) for SHOT in [0, 1, 2, 3, 4, 5]]
        for task_scores in scores:
            assert len(task_scores) == len(TASK_TO_NAME)
        score_avg = [sub_score for shot_scores in scores for sub_score in shot_scores]
        all_scores.append(np.mean(score_avg)*100)
    scores_avg_str_c4.append(f"{round(np.mean(all_scores), 1)} ± {round(np.std(all_scores), 1)}")

"""
scores_avg_str_c4_nobabi = []
for model in MODEL_FAM:
    all_scores = []
    for model_name in [model+"", model+"seed1", model+"seed2", model+"seed3", model+"seed4"]:
        scores = [MODEL_TO_FEWSHOT_SCORES[MODEL_KEY][model_name][SHOT][task] for task in TASK_TO_NAME for SHOT in [0, 1, 2, 3, 4, 5] if task != "babi"]
        all_scores.append(np.mean(scores)*100)
    scores_avg_str_c4_nobabi.append(f"{round(np.mean(all_scores), 1)} ± {round(np.std(all_scores), 1)}")
"""

TABLE += "\nAverage & " + " & ".join(scores_avg_str_c4) + " \\\\"
#TABLE += "\nAverage (without bAbI) & " + " & ".join(scores_avg_str_c4_nobabi) + " \\\\"

TABLE += "\n\\bottomrule"


print("\n\n\n")
print(TABLE)










\begin{table}
  \centering
  \caption{2.8B perf.}
  \resizebox{\textwidth}{!}{
    \begin{tabular}{l|llllllll}
Unique Tokens & 1 & 2 & 3 & 4 & 5 & 7 & 14 & 44 \\
Epochs & 1 & 2 & 3 & 4 & 5 & 7 & 14 & 44 \\

ANLI R1 & -1.0 ± 0.3 & -0.7 ± 1.1 & -0.7 ± 1.0 & -0.4 ± 1.1 & 0.4 ± 0.8 & 0.5 ± 1.1 & 0.1 ± 0.9 & 0.2 ± 0.9 \\
ANLI R2 & 0.8 ± 0.5 & 0.8 ± 0.8 & 0.0 ± 1.4 & 0.5 ± 0.7 & 0.5 ± 0.9 & 0.3 ± 1.0 & 0.7 ± 0.7 & 2.5 ± 1.0 \\
ANLI R3 & 1.1 ± 0.7 & 0.8 ± 0.9 & 0.3 ± 0.8 & 1.4 ± 1.1 & 1.3 ± 0.9 & 2.3 ± 0.2 & 1.3 ± 0.2 & 1.6 ± 1.2 \\
ARC-Challenge & 5.3 ± 0.6 & 5.1 ± 0.9 & 5.2 ± 2.0 & 6.0 ± 0.8 & 4.7 ± 0.8 & 3.1 ± 0.4 & 2.9 ± 1.0 & -1.3 ± 1.0 \\
ARC-Easy & 49.2 ± 0.9 & 50.4 ± 1.2 & 47.4 ± 4.5 & 49.4 ± 0.7 & 48.7 ± 1.5 & 44.9 ± 0.7 & 45.0 ± 1.2 & 31.9 ± 0.9 \\
BoolQ & 18.2 ± 4.0 & 19.6 ± 5.1 & 22.1 ± 1.0 & 20.4 ± 3.6 & 18.4 ± 6.0 & 18.4 ± 3.9 & 18.9 ± 2.6 & -3.3 ± 7.1 \\
CB & 12.0 ± 7.2 & 8.5 ± 9.2 & 7.9 ± 10.4 & 19.6 ± 7.3 & 17.8 ± 7.3 & 15.1 ± 5.8 & 17.5 ± 3.5 & 19.5 ± 6.6 \\
COPA & 59.1

In [ ]:
### Latex Table ###

import numpy as np

TASK_TO_NAME = {
    "anli_r1": "ANLI R1",
    "anli_r2": "ANLI R2",
    "anli_r3": "ANLI R3",
    "arc_challenge": "ARC-Challenge",
    "arc_easy": "ARC-Easy",
    "boolq": "BoolQ",
    "cb": "CB",
    "copa": "COPA",
    "hellaswag": "HellaSwag",
    "piqa": "PiQA",
    "rte": "RTE",
    "sciq": "SciQ",
    "storycloze_2016": "StoryCloze 2016",
    "winogrande": "WinoGrande XL",

    "e2e_nlg_cleaned": "E2E NLG",
    "gem_xsum": "XSUM",
    "web_nlg_en": "WebNLG EN",
    "wiki_lingua_en": "WikiLingua EN",
    
    "babi": "bAbI",
}


TABLE = """
\\begin{table}
  \\centering
  \\caption{2.8B perf.}
  \\resizebox{\\textwidth}{!}{
    \\begin{tabular}{l|llllllll}
"""

MODEL_KEY = 'lm1-2b8-55b-oscar-repetitions' # Change to desired model
MODEL_KEY = 'lm1-4b2-84b-c4-repetitions' # Change to desired model

# MODEL_KEY = 'lm1-4b2-84b-c4-opt' # Not enough eval datasets to make a good comparison (too close to rndm); Loss is better to compare
MODEL_KEY = 'lm1-8b7-178b-oscar-repetitions' # Change to desired model
#MODEL_KEY2 = 'lm1-8b7-178b-oscar-repetitions' # Change to desired model

MODEL_FAM = list(MODELS_MAP[MODEL_KEY].values())[::-1]# + list(MODELS_MAP[MODEL_KEY2].values())[::-1]
MODEL_EPS = list(MODELS_MAP[MODEL_KEY].keys())[::-1]# + list(MODELS_MAP[MODEL_KEY2].keys())[::-1]

SUBHEADER = "Unique Tokens & " + " & ".join([str(x) for x in MODEL_EPS]) + " \\\\" + "\n"
SUBSUBHEADER = "Epochs & " + " & ".join([str(x) for x in MODEL_EPS]) + " \\\\" + "\n"

TABLE += SUBHEADER + SUBSUBHEADER

for task in TASK_TO_NAME:
    scores_c4 = []
    std_c4 = []
    for model in MODEL_FAM:
        if "seed" in model: continue
        model_scores = []
        for model_name in [model]:#+"", model+"seed1", model+"seed2", model+"seed3", model+"seed4"]:
            model_scores.append(sum([MODEL_TO_FEWSHOT_SCORES[MODEL_KEY][model_name][SHOT][task] for SHOT in [0, 1, 2, 3, 4, 5]]) / 6)
        scores_c4.append(np.mean(model_scores))
        std_c4.append(np.std(model_scores))

    #scores_c4 = [f"{str(round(score * 100, 1))} ± {str(round(std * 100, 1))}" for score, std in zip(scores_c4, std_c4)]
    scores_c4 = [f"{str(round(score * 100, 1))}" for score, std in zip(scores_c4, std_c4)]
    TABLE += "\n" + f"{TASK_TO_NAME[task]} & " + " & ".join(scores_c4) + " \\\\"

    if task in ("winogrande", "wiki_lingua_en", "babi"):
        TABLE += "\n" + "\\midrule"

# Add average
scores_avg_str_c4 = []
for model in MODEL_FAM:
    if "seed" in model: continue
    all_scores = []
    for model_name in [model]:#+"", model+"seed1", model+"seed2", model+"seed3", model+"seed4"]:
        scores = [list(MODEL_TO_FEWSHOT_SCORES[MODEL_KEY][model_name][SHOT].values()) for SHOT in [0, 1, 2, 3, 4, 5]]
        for task_scores in scores:
            assert len(task_scores) == len(TASK_TO_NAME)
        score_avg = [sub_score for shot_scores in scores for sub_score in shot_scores]
        all_scores.append(np.mean(score_avg)*100)
    scores_avg_str_c4.append(f"{round(np.mean(all_scores), 1)}")# ± {round(np.std(all_scores), 1)}")

"""
scores_avg_str_c4_nobabi = []
for model in MODEL_FAM:
    all_scores = []
    for model_name in [model+"", model+"seed1", model+"seed2", model+"seed3", model+"seed4"]:
        scores = [MODEL_TO_FEWSHOT_SCORES[MODEL_KEY][model_name][SHOT][task] for task in TASK_TO_NAME for SHOT in [0, 1, 2, 3, 4, 5] if task != "babi"]
        all_scores.append(np.mean(scores)*100)
    scores_avg_str_c4_nobabi.append(f"{round(np.mean(all_scores), 1)} ± {round(np.std(all_scores), 1)}")
"""

TABLE += "\nAverage & " + " & ".join(scores_avg_str_c4) + " \\\\"
#TABLE += "\nAverage (without bAbI) & " + " & ".join(scores_avg_str_c4_nobabi) + " \\\\"

TABLE += "\n\\bottomrule"


print("\n\n\n")
print(TABLE)










\begin{table}
  \centering
  \caption{2.8B perf.}
  \resizebox{\textwidth}{!}{
    \begin{tabular}{l|llllllll}
Unique Tokens & 1 & 2 & 3 & 4 & 5 & 7 & 14 & 44 \\
Epochs & 1 & 2 & 3 & 4 & 5 & 7 & 14 & 44 \\

ANLI R1 & -1.3 & -2.3 & -0.5 & -1.8 & 0.1 & -0.3 & 2.6 & -0.4 \\
ANLI R2 & 0.8 & 3.2 & -0.2 & -1.3 & 1.0 & 0.2 & 1.5 & 0.5 \\
ANLI R3 & 1.1 & 1.2 & 1.3 & 0.9 & 2.8 & -0.4 & 1.1 & -0.1 \\
ARC-Challenge & 6.9 & 6.7 & 6.9 & 3.8 & 6.6 & 4.8 & 4.0 & -0.9 \\
ARC-Easy & 50.2 & 51.6 & 51.2 & 51.0 & 51.9 & 50.8 & 47.0 & 33.0 \\
BoolQ & 18.4 & 11.7 & 19.4 & 22.4 & 17.5 & 20.8 & 7.6 & 4.1 \\
CB & 11.2 & 13.4 & 16.1 & 19.6 & 21.4 & 25.0 & 9.8 & 20.1 \\
COPA & 46.7 & 53.0 & 52.0 & 53.7 & 51.0 & 53.3 & 48.7 & 41.7 \\
HellaSwag & 27.4 & 27.2 & 26.8 & 26.8 & 27.3 & 26.7 & 25.5 & 19.6 \\
PiQA & 49.2 & 49.3 & 50.1 & 48.7 & 48.1 & 47.2 & 45.6 & 37.0 \\
RTE & -0.5 & 1.1 & 0.2 & 1.2 & 10.2 & 3.2 & -3.0 & -7.8 \\
SciQ & 88.1 & 88.0 & 88.4 & 87.9 & 87.9 & 87.4 & 86.3 & 64.6 \\
StoryCloze 2016 & 61.6 